# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2017, 8, 7, 10, 30, 21, 712256, tzinfo=datetime.timezone.utc), bid=1.1803, bidSize=3169000, ask=1.18035, askSize=14000000, high=1.1814, low=1.1771, close=1.1773)

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.1803249999999998

The following cell will start a 15 second loop that prints a live updated ticker table.
It is updated with every new network packet that arrives.

In [6]:
from IPython.display import display, clear_output
import pandas as pd
import time
runUntil = time.time() + 15

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]

while ib.waitOnUpdate():
    for i, t in enumerate(ib.tickers()):
        df.iloc[i, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
    clear_output(wait=True)
    display(df)
    if time.time() > runUntil:
        break

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,3169000,1.1803,1.18035,14000000,1.1814,1.1771,1.1773
1,USDJPY,24000000,110.83,110.84,10000000,110.895,110.645,110.69
2,GBPUSD,1000000,1.30525,1.3053,500000,1.30595,1.3036,1.3039
3,USDCHF,12700000,0.9723,0.97235,1000000,0.97465,0.97115,0.97285
4,USDCAD,13500000,1.2677,1.26775,500000,1.2683,1.26305,1.2652
5,AUDUSD,3252000,0.79105,0.7911,2752000,0.7949,0.79055,0.7928


After an update the tick data is available in the 'ticks' attribute of the ticker.
The tick data will be cleared before the next update.

In [7]:
import time
runUntil = time.time() + 15

while ib.waitOnUpdate():
    if ticker.ticks:
        print(ticker.ticks)
    if time.time() > runUntil:
        break

[TickData(time=datetime.datetime(2017, 8, 7, 10, 30, 50, 949628, tzinfo=datetime.timezone.utc), tickType=1, price=1.18025, size=19169000)]
[TickData(time=datetime.datetime(2017, 8, 7, 10, 30, 52, 934441, tzinfo=datetime.timezone.utc), tickType=2, price=1.1803, size=1000000)]
[TickData(time=datetime.datetime(2017, 8, 7, 10, 30, 54, 380689, tzinfo=datetime.timezone.utc), tickType=1, price=1.1803, size=1000000), TickData(time=datetime.datetime(2017, 8, 7, 10, 30, 54, 380689, tzinfo=datetime.timezone.utc), tickType=2, price=1.18035, size=24000000)]


To stop the live tick subscriptions:

In [8]:
for contract in contracts:
    ib.cancelMktData(contract)

In [9]:
ib.disconnect()